In [1]:
import requests
import pandas as pd
import re

In [2]:
response = requests.get('http://api.citybik.es/v2/networks/to-bike')

In [3]:
response.status_code
response.json()
data = response.json()


In [4]:
df = pd.DataFrame()
temp = {} 

In [5]:
stations = data['network']['stations']

In [6]:
for station in stations:
    name = re.sub(r'(\d*[.][ ])', '', station['name'])
    temp['Name'] = name
    temp['Latitude'] = station['latitude']
    temp['Longitude'] = station['longitude']
    temp['Free_bikes'] = station['free_bikes']
    temp['Empty_slots'] = station['empty_slots']
    temp['Status'] = station['extra']['status']
    df = pd.concat([df, pd.DataFrame(temp, index=[temp['Name']])], ignore_index=True)

In [7]:
df.head()

,Name,Latitude,Longitude,Free_bikes,Empty_slots,Status
0,Gerbido,45.044404,7.617624,5,3,offline
1,Fabbrichetta,45.067520,7.563057,5,3,offline
2,Borgo,45.056364,7.580996,5,3,offline
3,Villa Claretta,45.061715,7.581189,5,3,offline
4,Universita`,45.064792,7.588077,5,3,offline


In [8]:
print("Total number of Free Bikes: ", df['Free_bikes'].sum())
print("Total number of Empty Slots: ", df['Empty_slots'].sum())

Total number of Free Bikes:  104
Total number of Empty Slots:  94


In [9]:
df['Status'].value_counts()

offline    19
online      9
Name: Status, dtype: int64

In [10]:
from math import cos, acos, sin
def distance_coords(lat1, lng1, lat2, lng2):
    """Compute the distance among two points."""
    deg2rad = lambda x: x * 3.141592 / 180 
    lat1, lng1, lat2, lng2 = map(deg2rad, [ lat1, lng1, lat2, lng2 ])
    R = 6378100 # Radius of the Earth, in meters
    return R * acos(sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lng1 - lng2))
# Given Co-oridinates

In [11]:
# Given Co-oridinates
# lat, long = list(map(float, input("Enter the Latitude and Longitude: ").split()))
lat, long = 45.114829, 7.641313
# Find the nearest online station
min_dist = 100000
online_stations = df[df['Status'] == 'online']
for i in range(len(online_stations)):
    dist = distance_coords(lat, long, online_stations.iloc[i]['Latitude'], online_stations.iloc[i]['Longitude'])
    if dist < min_dist:
        min_dist = dist
        station = online_stations.iloc[i]['Name']

In [12]:
print("Nearest Online Station: ", station)
print("Distance: ", min_dist)

Nearest Online Station:  Gallo Praile
Distance:  0.0
